# Server Module

## 安裝PliotGaea Python Module

From pip install：

> ```shell
> pip install OViewPy
> ```
>

## 初始化Server物件
本章節為PliotGaea Python Module的起始點，在執行所有操作前須先初始化Server物件，提供Module取得Map Server資源的相關連線資訊。

In [ ]:
from OViewPy.server import Server

# 綁定Server物件
server = Server(url="http://127.0.0.1:8080")


綁定Server物件後，即可取得Server相關資訊。

In [ ]:
from OViewPy.server import Server

# 綁定Server物件
server = Server(url="http://127.0.0.1:8080")
# 取得目前Server版本
print("OView Map Server Version：", server.version)
# 取得WMTS URL
print("WMTS URL：", server.wmtsURL)
# 取得WMS URL
print("WMS URL：", server.wmsURL)
# 取得DoCommand URL
print("DoCommand URL：", server.docmdURL)


## 取得圖層列表

綁定Server物件後，可透過`getLayerList`取得2D圖層列表詳細資訊。<br/>
亦可使用`getOViewLayerList`取得3D圖層列表詳細資訊。

In [ ]:
from OViewPy.server import Server

# 綁定Server物件
server = Server(url="http://127.0.0.1:8080")
layerList = server.getLayerList()
OViewLayerList = server.getOViewLayerList()
for list in layerList:
    print(list)
print("===========================================")
for list in OViewLayerList:
    print(list)


## 刪除圖層

如果想要刪除Server中已存在的圖層，可透過`deleteLayer`與`deleteOViewLayer`分別刪除2D及3D圖層。
> 注意！此刪除無法復原，請警慎使用。

In [ ]:
from OViewPy.server import Server

# 綁定Server物件
server = Server(url="http://127.0.0.1:8080")
server.deleteLayer(layerName="ne_10m_lakes")
server.deleteOViewLayer(layerName="ModelSet")


## 上架圖層

### 將圖片上架至伺服器

透過`saveImageToServer`可將圖片(JPG、PNG等)上架至Server

> 需特別注意，上架的圖片需在Server端目錄底下，並有相對應的World File，且確認圖層名稱不是已存在的圖層。

| 參數名稱 | Type | 預設值 | 說明 |
| :-----: | :---: | :---: | :--: |
| imageFilePath | string | None | 圖片檔案位置 |
| layerName | string | None | 上架後的圖層名稱 |
| epsg | int | 4326 | 座標參考系統 |

In [ ]:
from OViewPy.server import Server
from OViewPy.layer import VectorLayer
from OViewPy.varstruct import GeoBoundary
from OViewPy.da import da

# 綁定Server物件
server = Server(url="http://127.0.0.1:8080")
# 綁定Layer物件
layer = VectorLayer(server=server, layerName="Town_MOI")
boundary = GeoBoundary(147522.218692, 2422004.773002,
                       351690.114369, 2813163.248085)
# 取得圖片，取得成功會回傳圖片bytes資料
img = layer.getMapImage(
    boundary=boundary,
    crs="EPSG:3826",
    width=512,
    height=512,
    format="image/png"
)
# 儲存圖片，並生成World File
da.saveImg(
    img=img,
    savePath=".",
    imgName="testImage",
    imgType="png",
    worldFile=True,
    boundary=boundary
)
# 將圖片上架至Server
server.saveImageToServer(
    imageFilePath="D:\\NCHCProject\\jupyter_notebook\\testImage.png",
    layerName="testImage",
    epsg=3826
)


執行結果如下：<br/>

![saveImageToServer](./img/saveImgToServer.png)

## 將Vector File上架至伺服器

透過`saveVectorFileToServer`可將Vector File上架至Server

> 需特別注意，上架的Vector File需在Server端目錄底下，並確認圖層名稱不是已存在的圖層。

| 參數名稱 | Type | 預設值 | 說明 |
| :-----: | :---: | :---: | :--: |
| VectorFilePath | string | None | Vector File位置 |
| layerName | string | None | 上架後的圖層名稱 |
| epsg | int | 4326 | 座標參考系統 |

In [5]:
from OViewPy.server import Server
from OViewPy.layer import VectorLayer
from OViewPy.varstruct import GeoBoundary
from OViewPy.da import da

# 綁定Server物件
server = Server(url="http://127.0.0.1:8080")
# 綁定Layer物件
map = VectorLayer(server=server, layerName="Town_MOI")
# 設定搜尋條件
sql = "County_ID==64"
# 取得向量資料
ret = map.getVectorEmtity(epsg=3826, sql=sql)
# 將向量資料存成SHP檔
da.saveAsShapeFile(sourceGeo=ret["geo"],
                   sourceAttr=ret["attr"], fileName="TestShp_Kh2")
# 將SHP檔上架至伺服器
server.saveVectorFileToServer(
    VectorFilePath="D:\\NCHCProject\\jupyter_notebook\\TestShp_Kh2.shp", layerName="TestShp_Kh2", epsg=3826)


Done！
